In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import pandas as pd
import sys
import os
from scipy import interpolate
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from scipy.optimize import curve_fit
import astropy.units as u
import astropy.constants as c

In [3]:
import libwwz

In [4]:
from astropy.timeseries import LombScargle

In [5]:
import stingray as sr
from stingray.events import EventList
from stingray.lightcurve import Lightcurve
from stingray import Lightcurve, Crossspectrum, sampledata,Powerspectrum,AveragedPowerspectrum
from stingray.simulator import simulator, models

In [5]:
help(libwwz)

Help on package libwwz:

NAME
    libwwz - Exposes the wwt function directly without import the wwz module.

PACKAGE CONTENTS
    plot_methods
    wwz

FILE
    /Users/baotong/opt/anaconda3/lib/python3.8/site-packages/libwwz/__init__.py




In [18]:
def get_hist(t, len_bin):
    ###将输入的time信息，按照len_bin的长度输出为lc
    t_test = t-t[0]
    a = [0 for i in range(int(t_test[-1] / len_bin) + 1)]
    for i in range(len(t_test)):
        a[int(t_test[i] / len_bin)] += 1
    a = np.array(a)
    return a

In [47]:
dt = 2  # seconds
long_dt = 2  # seconds
long_exposure = 160.  # seconds
long_times = np.arange(0, long_exposure, long_dt)  # seconds
# In count rate units here
long_signal = 100 * np.sin(2.*np.pi*long_times/10) + 1000

# Multiply by dt to get count units, then add Poisson noise
long_noisy = np.random.poisson(long_signal*long_dt)

In [48]:
long_lc = Lightcurve(long_times, long_noisy)

In [49]:
help(libwwz.wwt)

Help on function wwt in module libwwz.wwz:

wwt(timestamps: numpy.ndarray, magnitudes: numpy.ndarray, time_divisions: int, freq_params: list, decay_constant: float, method: str = 'linear', parallel: bool = True) -> numpy.ndarray
    The code is based on G. Foster's FORTRAN
    code as well as eaydin's python 2.7 code. The code is updated to use numpy methods and allow for float value tau.
    It returns an array with matrix of new evenly spaced timestamps, frequencies, wwz-power, amplitude, coefficients,
    and effective number. Specific equations can be found on Grant Foster's "WAVELETS FOR PERIOD ANALYSIS OF UNEVENLY
    SAMPLED TIME SERIES". Some of the equations are labeled in the code with corresponding numbers.
    
    :param timestamps: An array with corresponding times for the magnitude (payload).
    :param magnitudes: An array with payload values
    :param time_divisions: number of divisions for the new timestamps
    :param freq_params: A list containing parameters for ma

In [50]:
freq=np.arange(1/100.,1./5,1e-2)

In [59]:
def generalized_lorentzian(x, p):
    """
    Generalized Lorentzian function.

    Parameters
    ----------

    x: numpy.ndarray
        non-zero frequencies

    p: iterable
        p[0] = peak centeral frequency
        p[1] = FWHM of the peak (gamma)
        p[2] = peak value at x=x0
        p[3] = power coefficient [n]

    Returns
    -------
    model: numpy.ndarray
        generalized lorentzian psd model
    """

    assert p[3] > 0., "The power coefficient should be greater than zero."
    return p[2] * (p[1] / 2)**p[3] * 1./(abs(x - p[0])**p[3] + (p[1] / 2)**p[3])
def smoothbknpo(x, p):
    """
    Smooth broken power law function.

    Parameters
    ----------

    x: numpy.ndarray
        non-zero frequencies

    p: iterable
        p[0] = normalization frequency
        p[1] = power law index for f --> zero
        p[2] = power law index for f --> infinity
        p[3] = break frequency

    Returns
    -------
    model: numpy.ndarray
        generalized smooth broken power law psd model
    """

    return p[0] * x**(-p[1]) / (1. + (x / p[3])**2)**(-(p[1] - p[2]) / 2)

In [70]:
def sim_evtlist(lc):
    num_evt_bin=np.random.poisson(lc.counts)
    evt_all=[]
    i=0
    while i < len(num_evt_bin):
        evt_all=np.concatenate((evt_all,(np.random.uniform(lc.time[i]-lc.dt/2,lc.time[i]+lc.dt/2,num_evt_bin[i]))))
        i+=1
    return evt_all

In [108]:
exptime=100000
tstart=12213.
dt = 50
cts_rate = 1e-1 * dt  # 实际的cts-rate应为这个的2倍
num_bins = int(exptime / dt)
sim = simulator.Simulator(N=num_bins, mean=cts_rate, dt=dt)
w = np.arange(1 / exptime, 0.5 / dt, 1 / exptime)
# w = np.fft.rfftfreq(sim.N, d=sim.dt)[1:]
# spectrum=generalized_lorentzian(w,[1/500.,1/4000.,100,2])
spectrum = smoothbknpo(w, [cts_rate*dt, 0.2, 1, 1e-4])  + generalized_lorentzian(w, [1 / 1000., 1 / 4000., cts_rate*dt*10, 2])
lc = sim.simulate(spectrum)
lc.counts += cts_rate
lc.counts[np.where(lc.counts < 0)] = 0
T_exp =  exptime

ev = EventList()
# ev.simulate_times(use_spline=False,lc=lc,bin_time=dt)
ev.time=sim_evtlist(lc)+tstart
print(len(ev.time))
# print(2*cts_rate*num_bin
# ev_all=ev_all.join(ev)        
lc_new = ev.to_lc(dt=dt, tstart=ev.time[0]-0.5*dt, tseg=ev.time[-1]-ev.time[0])
# plt.figure(2)
# plt.plot(lc_new.time,lc_new.counts,c='r')
# plt.show()
# plt.figure(3)
T_exp=lc_new.time[-1]-lc_new.time[0]
freq=np.arange(1/T_exp,0.5/dt,1/T_exp)
freq=freq[np.where(freq > 1 / 3000.)]

print(len(freq))

20228
965


/Users/baotong/opt/anaconda3/lib/python3.8/site-packages/stingray/utils.py:118: UserWarning: SIMON says: Stingray only uses poisson err_dist at the moment. All analysis in the light curve will assume Poisson errors. Sorry for the inconvenience.
  warnings.warn("SIMON says: {0}".format(message), **kwargs)


In [109]:
[Tau, Freq, WWZ, AMP, COEF, NEFF]=libwwz.wwt(lc_new.time,lc_new.counts,time_divisions=1000,freq_params=[freq[0],freq[-1],freq[-1]-freq[-2],False],decay_constant=1e-3)

*** Starting Weighted Wavelet Z-transform ***

Pseudo sample frequency (median) is  0.02
largest tau window is  99.95
565.42 seconds has passed to complete Weighted Wavelet Z-transform 



In [19]:
bin_len=100.
path='/Users/baotong/Desktop/CDFS/txt_all_obs_0.5_8_ep3/'
filename='127.txt'
time=np.loadtxt(path+filename)[:,0]
T_exp = time[-1] - time[0]
freq=np.arange(1/T_exp,0.5/bin_len,1/(5*T_exp))            
flux=get_hist(time,bin_len)
x=np.arange(bin_len/2.,(time[-1]-time[0])+bin_len/2.,bin_len)

In [ ]:
[Tau, Freq, WWZ, AMP, COEF, NEFF]=libwwz.wwt(timestamps=x,magnitudes=flux,time_divisions=1000,freq_params=[freq[0],freq[-1],freq[-1]-freq[-2],False],decay_constant=1e-3)

*** Starting Weighted Wavelet Z-transform ***

Pseudo sample frequency (median) is  0.01
largest tau window is  10989.489


In [110]:
%matplotlib widget      
fig, ax = plt.subplots()
ax.set_title('test title')
libwwz.plot_methods.linear_plotter(ax,Tau,Freq,WWZ)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1]:
path='/Users/baotong/Desktop/CDFS/'
file1='145108.76+270926.9_lccorr_1000_soft.fits'
file2='145108.76+270926.9_lccorr_1000_soft_2nd.fits'

In [ ]:
fits.open(path+file1)[1].data